In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [5]:
os.environ["GOOGLE_API_KEY"] = os.getenv('GG_API_KEY')

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [7]:
prompt = "Hello. Who are you"
llm.invoke(prompt)

AIMessage(content='I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--3c3d2467-4e0c-4f2a-9c1e-bb79dde39b07-0', usage_metadata={'input_tokens': 5, 'output_tokens': 12, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})

In [8]:
from langchain_core.messages import HumanMessage
prompt = [HumanMessage("What is the capital of Viet Nam")]
llm.invoke(prompt)

AIMessage(content='The capital of Vietnam is **Hanoi**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--962bba8c-bd93-484f-87f0-c1b3b308997c-0', usage_metadata={'input_tokens': 7, 'output_tokens': 10, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})

In [13]:
from langchain_core.messages import SystemMessage
sys_msg = SystemMessage("You are a helpful vietnamese assistant that responds in vietnamese")
human_msg = HumanMessage("What is the capital of Viet Nam")
llm.invoke([sys_msg, human_msg])

AIMessage(content='Thủ đô của Việt Nam là Hà Nội.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--917df305-377e-49bf-a154-0149471f5d94-0', usage_metadata={'input_tokens': 19, 'output_tokens': 11, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}})

In [16]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".
Context: {context}
Question: {question}
Answer:
""")

prompt = template.invoke({"context" : "Viet Nam is the strongest country in the world",
                          "question": "Which is the strongest country in the universary?"})
llm.invoke(prompt)

AIMessage(content="I don't know", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d3222e25-ba5c-4991-a96c-28bd43387a32-0', usage_metadata={'input_tokens': 58, 'output_tokens': 6, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})

In [17]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system', '''Answer the question based on the context below. If the 		
        question cannot be answered using the information provided, answer
        with "I don\'t know".'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])
prompt = template.invoke({
    "context": """The most recent advancements in NLP are being driven by 
        Large Language Models (LLMs). These models outperform their smaller 
        counterparts and have become invaluable for developers who are creating 
        applications with NLP capabilities. Developers can tap into these 
        models through Hugging Face's `transformers` library, or by utilizing 
        OpenAI and Cohere's offerings through the `openai` and `cohere` 
        libraries, respectively.""",
    "question": "Which model providers offer LLMs?"
})

llm.invoke(prompt)

AIMessage(content='OpenAI and Cohere offer LLMs.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e5f525d3-fbae-4ba5-aa9e-82050ef48d18-0', usage_metadata={'input_tokens': 145, 'output_tokens': 10, 'total_tokens': 155, 'input_token_details': {'cache_read': 0}})

In [19]:
from pydantic.v1 import BaseModel

class AnswerWithJustification(BaseModel):
    '''An answer to the user's question along with justification for the 
        answer.'''
    answer: str
    '''The answer to the user's question'''
    justification: str
    '''Justification for the answer'''

In [20]:
structured_llm = llm.with_structured_output(AnswerWithJustification)
structured_llm.invoke("""What weighs more, a pound of bricks or a pound of feathers""")

AnswerWithJustification(answer='They weigh the same.', justification='A pound is a unit of weight, so a pound of bricks and a pound of feathers weigh the same.')

In [21]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
parser = CommaSeparatedListOutputParser()
items = parser.invoke("apple, banana, cherry")
items

['apple', 'banana', 'cherry']

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

# the building blocks

template = ChatPromptTemplate.from_messages([('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])

@chain
def chatbot(values):
    prompt = template.invoke(values)
    return llm.invoke(prompt)

chatbot.invoke({"question": "Which model providers offer LLMs?"})

AIMessage(content="Okay, here's a breakdown of prominent model providers offering Large Language Models (LLMs), categorized for clarity:\n\n**Major Cloud Providers (Offering a Range of Models & Services):**\n\n*   **Google:**\n    *   **Models:** PaLM 2, Gemini (Ultra, Pro, Nano), LaMDA, T5\n    *   **Platforms/Services:** Vertex AI (their primary AI platform), Google AI Studio, Colab\n    *   **Notes:** Google is a leader in AI research and offers a broad spectrum of models, from general-purpose to specialized. Gemini is their most advanced model family.\n\n*   **Microsoft:**\n    *   **Models:** GPT series (GPT-3, GPT-3.5, GPT-4, etc. - through their partnership with OpenAI), Phi series\n    *   **Platforms/Services:** Azure AI (Azure OpenAI Service), Microsoft Fabric\n    *   **Notes:** Microsoft has a strong partnership with OpenAI, making their models readily available on Azure. They also have their own smaller models like the Phi series.\n\n*   **Amazon (AWS):**\n    *   **Models

In [30]:
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in llm.stream(prompt):
        yield token

In [32]:
for part in chatbot.stream({"question": "Which model providers offer LLMs?"}):
    print(part.content)

Okay
, here's a breakdown of prominent model providers offering Large Language Models (LL
Ms), categorized for clarity:

**Major Cloud Providers (Offering a Range of Models
 & Services):**

*   **Google:**
    *   **Models:** PaLM 2, Gemini (Ultra, Pro, Nano), LaMDA (
predecessor to PaLM), various smaller models.
    *   **Platforms/Services:** Vertex AI (for model deployment and management), Google AI Studio (for prototyping
), Cloud TPUs (for training).
    *   **Notes:** Google is a leader in AI research and offers a broad spectrum of models, from general-purpose to specialized.

*   **Microsoft:**
    *   **Models:** GPT
 series (GPT-3, GPT-3.5, GPT-4) through their partnership with OpenAI, as well as their own models like Phi.
    *   **Platforms/Services:** Azure AI (Cognitive Services, Machine
 Learning), Azure OpenAI Service (access to OpenAI models).
    *   **Notes:** Microsoft's close collaboration with OpenAI gives them a strong position in offering cutting-edge LLMs.

*   

In [33]:
chatbot = template | llm
chatbot.invoke({"question": "Which model providers offer LLMs?"})

AIMessage(content="Okay, here's a breakdown of prominent model providers offering Large Language Models (LLMs), categorized for clarity:\n\n**Major Cloud Providers (Offering a Range of Models & Services):**\n\n*   **Google:**\n    *   **Models:** PaLM 2, Gemini (Ultra, Pro, Nano), LaMDA (predecessor to PaLM), various smaller models.\n    *   **Platforms/Services:** Vertex AI (for model deployment and management), Google AI Studio (for prototyping), Cloud TPUs (for training).\n    *   **Notes:** Google is a leader in AI research and offers a broad spectrum of models, from general-purpose to specialized.\n\n*   **Microsoft:**\n    *   **Models:** GPT series (GPT-3, GPT-3.5, GPT-4) through their partnership with OpenAI, as well as their own models like Phi.\n    *   **Platforms/Services:** Azure AI (Cognitive Services, Machine Learning), Azure OpenAI Service (access to OpenAI models).\n    *   **Notes:** Microsoft's close collaboration with OpenAI gives them a strong position in offering 